In [1]:
from src import Tensor,cross_entropy,Adam,SGD,Linear, Module, Sequential, ReLU,dataloader,Conv2D
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import sklearn.datasets
train_X, train_y = sklearn.datasets.load_digits(return_X_y=True)

In [3]:
class Net(Module):
    def __init__(self, outc, impl):
        super().__init__()
        self.seq = Sequential(
            Conv2D(None, 64, 3, conv_impl=impl),
            ReLU(),
            lambda x: x.reshape(x.shape[0], -1),
            Linear(None, outc,bias=False)
        )

    def forward(self, x):
        x = self.seq(x)
        x = x.reshape(x.shape[0],-1)
        return x


def train(net,optim,train_X,train_y,epochs):
    if train_X.ndim == 2:
        h=w=int(np.sqrt(train_X.shape[1]))
        train_X = train_X.reshape(train_X.shape[0],1,h,w)
    import time
    start = time.monotonic()
    for epoch in range(epochs):  # loop over the dataset multiple times
        losses = []
        accs = []
        for X, y in dataloader(64*2, train_X, train_y):
            # forward + backward + optimize
            outputs = net.forward(X)
            loss = cross_entropy(outputs, y)
            optim.zero_grad()
            loss.backward()
            assert loss.item() >= 0, loss
            optim.step()

            losses.append(loss.item())
            accs.append((outputs.data.argmax(-1) == y.data).astype(float).mean())
        print(
            f'[Epoch {epoch + 1:3}] loss: {np.mean(losses):.3f} accuracy: {np.mean(accs)*100:3.2f}')
    return time.monotonic() - start

In [4]:
net = Net(np.unique(train_y).size, impl="slow")
optimizer = Adam(net.trainable_params(), lr=0.001, amsgrad=True)
time = train(net, optimizer, train_X, train_y, 10)
print(f'Finished Training in : {time:.4f} s')

[Epoch   1] loss: 43.823 accuracy: 58.25
[Epoch   2] loss: 9.948 accuracy: 84.06
[Epoch   3] loss: 3.067 accuracy: 94.06
[Epoch   4] loss: 1.274 accuracy: 97.40
[Epoch   5] loss: 0.793 accuracy: 98.02
[Epoch   6] loss: 0.561 accuracy: 98.91
[Epoch   7] loss: 0.389 accuracy: 97.78
[Epoch   8] loss: 1.785 accuracy: 94.76
[Epoch   9] loss: 2.008 accuracy: 95.62
[Epoch  10] loss: 0.626 accuracy: 97.86
Finished Training in : 14.9060 s


In [5]:
net = Net(np.unique(train_y).size, impl="fast_forward")
optimizer = Adam(net.trainable_params(), lr=0.001, amsgrad=True)
time = train(net, optimizer, train_X, train_y, 10)
print(f'Finished Training in : {time:.4f} s')

[Epoch   1] loss: 60.042 accuracy: 47.29
[Epoch   2] loss: 12.752 accuracy: 84.45
[Epoch   3] loss: 3.336 accuracy: 94.32
[Epoch   4] loss: 1.958 accuracy: 95.59
[Epoch   5] loss: 3.038 accuracy: 94.34
[Epoch   6] loss: 1.974 accuracy: 95.47
[Epoch   7] loss: 1.305 accuracy: 96.69
[Epoch   8] loss: 1.226 accuracy: 96.77
[Epoch   9] loss: 0.608 accuracy: 98.39
[Epoch  10] loss: 0.351 accuracy: 99.01
Finished Training in : 11.1720 s


In [6]:
net = Net(np.unique(train_y).size, impl="fast")
optimizer = Adam(net.trainable_params(), lr=0.001, amsgrad=True)
time = train(net, optimizer, train_X, train_y, 10)
print(f'Finished Training in : {time:.4f} s')

[Epoch   1] loss: 53.737 accuracy: 47.24
[Epoch   2] loss: 8.990 accuracy: 88.88
[Epoch   3] loss: 3.778 accuracy: 93.02
[Epoch   4] loss: 1.675 accuracy: 96.51
[Epoch   5] loss: 0.988 accuracy: 97.66
[Epoch   6] loss: 2.129 accuracy: 97.05
[Epoch   7] loss: 1.704 accuracy: 95.33
[Epoch   8] loss: 2.086 accuracy: 95.10
[Epoch   9] loss: 0.918 accuracy: 96.64
[Epoch  10] loss: 2.431 accuracy: 96.15
Finished Training in : 2.8130 s
